## Deliverable 2. Create a Customer Travel Destinations Map.

In [71]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys
# Import API key
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [72]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Yellowknife,CA,62.4560,-114.3525,-6.59,90,84,5.99,broken clouds
1,1,Taoudenni,ML,22.6783,-3.9836,69.08,11,100,12.24,overcast clouds
2,2,Amahai,ID,-3.3333,128.9167,84.96,67,100,5.48,overcast clouds
3,3,Luderitz,NaN,-26.6481,15.1594,67.14,73,0,22.46,clear sky
4,4,Kuloy,RU,61.0305,42.4925,14.09,95,100,7.14,light snow


In [73]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [74]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Amahai,ID,-3.3333,128.9167,84.96,67,100,5.48,overcast clouds
9,9,Rikitea,PF,-23.1203,-134.9692,78.03,71,90,13.89,overcast clouds
11,11,Kudahuvadhoo,MV,2.6708,72.8944,80.31,64,40,10.29,scattered clouds
16,16,Kapaa,US,22.0752,-159.3190,77.11,59,34,1.01,scattered clouds
22,22,Ahuimanu,US,21.4447,-157.8378,77.94,73,3,1.99,clear sky
24,24,Hilo,US,19.7297,-155.0900,75.25,85,4,4.00,clear sky
28,28,Kahului,US,20.8947,-156.4700,79.86,71,9,7.47,clear sky
29,29,Vaini,TO,-21.2000,-175.2000,82.78,78,100,17.07,overcast clouds
43,43,Poum,NC,-20.2333,164.0167,83.07,73,80,9.75,broken clouds
54,54,Carnarvon,AU,-24.8667,113.6333,83.53,61,1,12.59,clear sky


In [75]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                148
City                   148
Country                148
Lat                    148
Lng                    148
Max Temp               148
Humidity               148
Cloudiness             148
Wind Speed             148
Current Description    148
dtype: int64

In [76]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [77]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Amahai,ID,84.96,overcast clouds,-3.3333,128.9167,
9,Rikitea,PF,78.03,overcast clouds,-23.1203,-134.9692,
11,Kudahuvadhoo,MV,80.31,scattered clouds,2.6708,72.8944,
16,Kapaa,US,77.11,scattered clouds,22.0752,-159.3190,
22,Ahuimanu,US,77.94,clear sky,21.4447,-157.8378,
24,Hilo,US,75.25,clear sky,19.7297,-155.0900,
28,Kahului,US,79.86,clear sky,20.8947,-156.4700,
29,Vaini,TO,82.78,overcast clouds,-21.2000,-175.2000,
43,Poum,NC,83.07,broken clouds,-20.2333,164.0167,
54,Carnarvon,AU,83.53,clear sky,-24.8667,113.6333,


In [78]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [79]:
# 7. Drop the rows where there is no Hotel 
import numpy as np

hotel_df["Hotel Name"].replace("", np.nan, inplace = True)

clean_hotel_df = hotel_df.dropna()

In [80]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [89]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [90]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))